In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Latex



den_csv = 'MMS2_ni.csv'
mag_arr_str = 'MMS2_B.csv'
v_arr_str = 'MMS2_Vi.csv'

temp_perp = 'MMS2_Tiperp.csv'

temp_para = 'MMS2_Tipara.csv'



mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])
v_arr = pd.read_csv(v_arr_str,header = 0,names = ['times','Vix','Viy','Viz','date'],
                         parse_dates=['date'])

den_csv = pd.read_csv(den_csv,header = 0,names = ['times','Ni','date'],
                         parse_dates=['date'])



Tipara_csv = pd.read_csv(temp_para,header = 0,names = ['times','Tipara','date'],
                         parse_dates=['date'])

Tiperp_csv = pd.read_csv(temp_perp,header = 0,names = ['times','Tiperp','date'],
                         parse_dates=['date'])

In [2]:
def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return int(idx)

In [3]:
#Constants

kb = 1.38066*1e-23     #Boltzmann constant (J/K) (kg m^2 s^-2)(K^-1)
μ0 = 1.256637061*1e-6 # N A^-2 #kg m s^-2 A^-2






# indices in B-field array that correspond to timestamps in Temperature array

closest_indices_b = []

B_times = np.array(mag_arr['times'])
Tipara_times = np.array(Tipara_csv['times'])


for t in range(len(Tipara_times)):
    idx = find_nearest(B_times, Tipara_times[t])
    closest_indices_b.append(idx)


 

In [4]:
B_GSE = np.array(mag_arr['Bmag'][closest_indices_b])

Tipara = np.array(Tipara_csv['Tipara'])*11606 # T in eV convert to K
Tiperp= np.array(Tiperp_csv['Tiperp'])*11606

# define alphas based on Paschmann et al., 1986

P_para = (den_csv['Ni'])*1e6*kb*Tipara # P in units of Pa (m^-3)(kg m^2 s^-2 K^-1)(K)

P_perp = (den_csv['Ni'])*1e6*kb*Tiperp



# # alpha for plotting:
α = ((P_para - P_perp)*μ0)/(B_GSE*1e-9)**2


In [5]:
alpha = pd.DataFrame({"times": Tipara_times, "alpha": α , "date": Tipara_csv['date']})


In [6]:
alpha.to_csv('pressure_anisotropy.csv', index=False)  